## scraping part

In [1]:
import json
from datetime import datetime
from BAScraper import Pushpull
from os import path

cwd = '../results'
data_path = path.join(cwd, 'data.json')

pp = Pushpull(sleepsec=3, threads=4, cwd=cwd)

data = pp.get_submissions(after=datetime(2024, 1, 1), before=datetime(2024, 1, 2),
                          subreddit='bluearchive', get_comments=True, duplicate_action='keep_original')

with open(data_path, "w", encoding='utf-8') as outfile:
    json.dump(data, outfile, indent=4)

INFO - started thread no.0 | 2024-01-01 06:00:00 <- 2024-01-01 00:00:00


no CSE credentials detected. you'll need it to use the Google Custom Search Engine


INFO - started thread no.1 | 2024-01-01 12:00:00 <- 2024-01-01 06:00:00
INFO - started thread no.2 | 2024-01-01 18:00:00 <- 2024-01-01 12:00:00
INFO - started thread no.3 | 2024-01-02 00:00:00 <- 2024-01-01 18:00:00
INFO - t-1: 200 - 0:00:00.942073
INFO - t-2: 200 - 0:00:00.932266
INFO - t-0: 200 - 0:00:01.257051
INFO - t-3: 200 - 0:00:00.957824
INFO - t-1: 200 - 0:00:01.105028
INFO - t-2: 200 - 0:00:00.568458
INFO - t-0: 200 - 0:00:00.432588
INFO - t-3: 200 - 0:00:00.865668
INFO - t-1: finished.
INFO - t-2: finished.
INFO - t-0: finished.
INFO - t-3: finished.
INFO - submission fetching time: 12.53048825263977sec
INFO - dupe detected for 18vwza2
INFO - dupe detected for 18vwws2
INFO - dupe detected for 18vszv8
INFO - dupe detected for 18vs7jk
INFO - dupe detected for 18vqn02
INFO - dupe detected for 18vqb9b
INFO - dupe detected for 18vqaq8
INFO - dupe detected for 18vo28d
INFO - dupe detected for 18vllwi
INFO - dupe detected for 18vll0o
INFO - dupe detected for 18vlkms
INFO - dupe det

# EDA

In [2]:
import pandas
from collections import Counter
from os import path
import json
from datetime import datetime

cwd = '../results'
data_path = path.join(cwd, 'data.json')
student_path = path.join(cwd, 'students.txt')

exclude_author = ['AutoModerator', 'BlueArchive-ModTeam']

with open(data_path, "r", encoding='utf-8') as outfile:
    data = json.load(outfile)

data_df = pandas.DataFrame.from_dict(data, orient='index')
# [print(col) for col in data_df.columns]
filtered_df = data_df[['title', 'link_flair_text', 'ups', 'created_utc', 'comments', 'num_comments']]
filtered_df.created_utc = filtered_df.created_utc.apply(lambda x: datetime.fromtimestamp(x))
filtered_df.comments = filtered_df.comments.apply(lambda x: [comment['body'] for comment in x if comment['author'] not in exclude_author])
filtered_df.num_comments = filtered_df.comments.apply(lambda x: len(x))
filtered_df = filtered_df.loc[filtered_df['link_flair_text'].isin(['OC ART ', 'NON OC ART ', 'Comic/TL'])]

display(filtered_df.nlargest(10, 'num_comments'))
Counter(filtered_df.link_flair_text)

/tmp/ipykernel_31744/3429226781.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas
/tmp/ipykernel_31744/3429226781.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.created_utc = filtered_df.created_utc.apply(lambda x: datetime.fromtimestamp(x))
/tmp/ipykernel_31744/3429226781.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

,title,link_flair_text,ups,created_utc,comments,num_comments
18vokb1,Hoshino's Intrusive Thoughts Leaked (Translate...,Comic/TL,1,2024-01-01 13:03:14,[Don't send help I'm perfectly happy here ![im...,45
18vju8z,"I'm going insane again (Mika, Tsurugi, Hina)",OC ART,1,2024-01-01 08:25:46,"[fire.., Hina scars 🥵, For real, I don't get i...",44
18vnj59,New Year Mutsuki (by @12beeeef),NON OC ART,1,2024-01-01 11:57:06,"[nah, nai v3 can already do halos in different...",41
18vdmi2,"New Year's Blessing be upon you! [Plana, Arona...",Comic/TL,1,2024-01-01 03:19:10,"[I'll take your feet plana-chan, Don’t even jo...",39
18vw375,"""Punishment Time!"" - Nonomi versus heavy armor...",NON OC ART,1,2024-01-01 21:47:06,"[The others wouldn't let her, It's been so lon...",34
18vo28d,I drew Shizuko :3,OC ART,1,2024-01-01 12:30:36,"[Noooooo😭😭😭, Nooooooo, 🔥🔥🔥✍️✍️✍️🔥🔥🔥, Hot, To P...",30
18vvs4q,Swimsuit Valkyrie mob-chan in a bind (by @HuDe...,NON OC ART,1,2024-01-01 21:26:15,"[So true brother, honestly i like the more ado...",29
18vu04w,Miyu and... Mr Seagull? (By @ylgcoffee),NON OC ART,1,2024-01-01 19:20:56,"[Mr Barry Chopsticks the seagull, Sir, don't d...",28
18voxc0,"Happy New Year, fellow Sensei!!! Ready for ano...",OC ART,1,2024-01-01 13:26:50,[Worst pulls? S. Hoshino banner. = sparked\n\n...,28
18vn9lc,"New Year Pocket Money [Himari, Momoi, Midori] ...",Comic/TL,1,2024-01-01 11:40:30,"[Wow, rude, First and last time that Sensei al...",25


Counter({'NON OC ART ': 38, 'OC ART ': 26, 'Comic/TL': 8})

**Note that the title for the post is appended to the start of the comment doc group**

In [15]:
import re
import html

sub_emote_pattern = r"!\[img\]\(emote\|t5_[a-z0-9]+\|\d+\)"
url_pattern = r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"

patterns = [sub_emote_pattern, url_pattern]

def clean(text):
    for pattern in patterns:
        text = re.sub(pattern, '', text)
        text.replace('senseis', 'senseis') # problems with the senseis plural in the model
    return html.unescape(text).lower()

data_text = { i: [v.title] + v.comments for i, v in filtered_df.iterrows()}
data_text = { i: clean('\n'.join(post_group)) for i, post_group in data_text.items()}

data_text['18vokb1']

'hoshino\'s intrusive thoughts leaked (translated) [kuuload]\ndon\'t send help i\'m perfectly happy here \n\nmom\'s spaghetti\nthat sounds like some slaneeshi demon to me.\npoor sleepy oji-san\nbased\nsensei cant leave if he cant walk,  \n\nproceeds to blow of senseis leg with her shotgun\ni won\'t go anywhere without my beloved uncle. \ni have no idea what that is and i have a feeling i don\'t wanna know \n r/japanpeopletwitter moment.\nhoshino i\'m gonna be ok. you can go rest now.\nmom sketty vomit sweater ready *n e r v o u s*\nin this state, hoshino\'s ahoge is the only part of her going atsui.\nfastest uoogh in kivotos\nduring an official comic she immediately masked up as mask number 0 of the masked swimsuit gang when she thought hifumi was asking them to rob a bank.\nit actually showed during volume f about ayane getting peer pressured into doing something stupid.\ni don\'t mind having another mentally unhinged stalker follow me around and keep me safe. and sadly i will not be 

In [16]:
import spacy
from spacy.tokens import Token
from spacymoji import Emoji

print(spacy.require_gpu())

student_list = list()
with open(student_path, 'r', encoding='utf-8') as f:
    for l in f.readlines():
        student_list.append(l.strip())

# alias groups - (main name, alias-1, alias-2, alias-3, ...)
alias_groups = [
    ('aris', 'alice', 'arisu', 'fridge', 'refrigerator'),
    ('hoshino', 'oji-san', 'ojisan', 'oji san'),
    ('yuuka', 'yuka', 'calculator', '100kg', '100 kg'),
    # ('sensei', 'senseis'), # cannot be detected as plural
]

student_list.extend([j for i in alias_groups for j in i])

print('student_list:', len(student_list))
print(student_list)


def BA_char_finder(tk):
    if (student := tk.text.lower()) in student_list:
        for group in alias_groups:
            if student in group:
                student = group[0]
        return student
    else:
        return None


def get_nbors_(tk: spacy.tokens.token.Token, step: int, N: int):
    # step usually should be 1 or -1 - searching range step
    # N is the Nth search result to be returned

    if tk.is_sent_start and step < 0:
        return
    elif tk.is_sent_end and step > 0:
        return
    
    n = 1
    i = step
    while True:
        nbor = tk.nbor(i)
        if nbor.is_stop or nbor._.is_emoji or not nbor.is_alpha:
            pass
        else:
            if n == N:
                return nbor
            else:
                n += 1
                pass
        
        if nbor.is_sent_end or nbor.is_sent_start:
            return

        i += step


def get_nbors(tk):
    return ((
        (
        get_nbors_(tk, -1, 1), # l1
        get_nbors_(tk, 1, 1), # r1
        ),
        (
        get_nbors_(tk, -1, 2), # l2
        get_nbors_(tk, 1, 2), # r2
        ),
        # tk.sent
    ))

# for splitting sentences
@spacy.language.Language.component("custom_boundary")
def custom_boundary(doc):

    delimiters=['...', '\n']

    for token in doc[:-1]:
        if token.text in delimiters:
            doc[token.i+1].is_sent_start = True
    
    return doc


nlp = spacy.load('en_core_web_lg')
nlp.add_pipe("emoji", first=True)
nlp.add_pipe('custom_boundary', before='parser')

Token.set_extension("nbors", getter=get_nbors, force=True)
Token.set_extension("ba_characters", getter=BA_char_finder, force=True)

analysis = nlp.analyze_pipes(pretty=True)

True
student_list: 176
['kayoko', 'sensei', 'master', 'arona', 'hare', 'nonomi', 'shokuhou', 'beatrice', 'iori', 'megu', 'meru', 'alice', 'kuzunoha', 'hifumi', 'yukari', 'tsurugi', 'nyanten-maru', 'phrenapates', 'saya', 'rumi', 'otogi', 'kaede', 'ibuki', 'iroha', 'eimi', 'karin', 'satsuki', 'akari', 'kai', 'sumomo', 'shiba', 'yuzu', 'makoto', 'wakamo', 'ayumu', 'haruna', 'saki', 'ruiko', 'rin', 'izumi', 'himari', 'niko', 'midori', 'ayane', 'aoi', 'gsc', 'erika', 'koharu', 'fubuki', 'kirino', 'takane', 'kisaki', 'akane', 'minori', 'hanae', 'maki', 'shinon', 'marina', 'decalcomanie', 'saten', 'reijo', 'sena', 'koyuki', 'reisa', 'renge', 'miku', 'mina', 'kaya', 'azusa', 'haruka', 'misaki', 'golconde', 'hibiki', 'mashiro', 'suzumi', 'shiroko', 'plana', 'maestro', 'kotama', 'mikoto', 'tomoe', 'sora', 'serina', 'umika', 'nagisa', 'sakurako', 'ako', 'momoka', 'niya', 'izuna', 'kurumi', 'junko', 'hiyori', 'shimiko', 'mimori', 'nodoka', 'atsuko', 'aru', 'kirara', 'hina', 'michiru', 'chihiro', '

### planned pipeline

first, use the `set_extention` to set all the student attr. might add it as a custom pipeline. since the students have alternate names, nicknames I should consider that

**for getting co-occurrence matrix for students**
1. iter through the tokens
2. if `student` extension returns a student: start the below, else: continue
3. get the neighboring words (maybe a range of 2 on each side, and give the closer one higher score)
4. if the neighbor token: is a stopword(`is_stop`): pass, is an emoji(`is_emoji`): pass
5. expand the search range until the required number of token(2 on each side) is found
6. lowercase (and probably clean up a bit) the token and add to list with required metadata(such as score)
7. score for each descriptive token is determined using TF-IDF

### stuffs to use

**token**
- get/set_extension
- nbor
- similarity

### testing pipeline

In [17]:
doc = nlp(data_text['18vnj59'])

In [18]:
for i, tk in enumerate(doc):
    print(f'{tk.lemma_} \t{tk.is_sent_start} / {tk.is_sent_end} - <{i}>')

new 	True / False - <0>
year 	False / False - <1>
mutsuki 	False / False - <2>
( 	False / False - <3>
by 	False / False - <4>
@12beeeef 	False / False - <5>
) 	False / False - <6>

 	False / True - <7>
nah 	True / False - <8>
, 	False / False - <9>
nai 	False / False - <10>
v3 	False / False - <11>
can 	False / False - <12>
already 	False / False - <13>
do 	False / False - <14>
halo 	False / False - <15>
in 	False / False - <16>
different 	False / False - <17>
position 	False / False - <18>
( 	False / False - <19>
even 	False / False - <20>
upside 	False / False - <21>
- 	False / False - <22>
down 	False / False - <23>
) 	False / False - <24>

 	False / True - <25>
seem 	True / False - <26>
like 	False / False - <27>
[ 	False / False - <28>
they 	False / False - <29>
get 	False / False - <30>
pretty 	False / False - <31>
damn 	False / False - <32>
close 	False / False - <33>
] 	False / False - <34>
( 	False / False - <35>
from 	False / False - <36>
what 	False / False - <37>
I 	False /

In [19]:
Counter([(emote[0], emote[2]) for emote in doc._.emoji]).most_common(10)

[(('😭', 'loudly crying face'), 52),
 (('💢', 'anger symbol'), 43),
 (('👺', 'goblin'), 18),
 (('😤', 'face with steam from nose'), 10),
 (('👹', 'ogre'), 8),
 (('👿', 'angry face with horns'), 7),
 (('😡', 'enraged face'), 6),
 (('😠', 'angry face'), 4),
 (('🚨', 'police car light'), 2),
 (('😳', 'flushed face'), 1)]

In [20]:
Counter([tk._.ba_characters for tk in doc if tk._.ba_characters])

Counter({'mutsuki': 11, 'sensei': 4, 'mika': 1, 'rin': 1})

In [21]:
[tk._.nbors for tk in doc if tk._.ba_characters == 'sensei']

[((greet, new), (panties, year)),
 ((temping, None), (brat, None)),
 ((teasing, like), (None, act)),
 ((plans, help), (derailing, None))]

## scoring (default embedding)


In [22]:
try: del doc
except: pass

from tqdm.notebook import tqdm

docs = list()
for t in tqdm(data_text.values()):
    docs.append(nlp(t))

  0%|          | 0/72 [00:00<?, ?it/s]

In [23]:
Counter([tk._.ba_characters for doc in docs for tk in doc if tk._.ba_characters]).most_common(10)

[('sensei', 91),
 ('arona', 31),
 ('yuuka', 27),
 ('himari', 24),
 ('hoshino', 23),
 ('hina', 22),
 ('plana', 22),
 ('rio', 17),
 ('mutsuki', 16),
 ('aru', 13)]

In [24]:
target_character = 'sensei'

# `cm` as in correlation matrix (not exactly a matrix but...)

cm_original = [tk._.nbors for doc in docs for tk in doc if tk._.ba_characters == target_character]
cm_original_1 = [j for i in cm_original for j in i[0] if j] # bigram range
cm_original_2 = [j for i in cm_original for j in i[1] if j] # trigram range

print(Counter([i.text for i in cm_original_1]).most_common(10))
print(Counter([i.text for i in cm_original_2]).most_common(10))

[('time', 5), ('like', 4), ('year', 4), ('fellow', 4), ('help', 3), ('times', 2), ('saw', 2), ('look', 2), ('needs', 2), ('thank', 2)]
[('year', 6), ('himari', 3), ('new', 3), ('kirino', 2), ('like', 2), ('squad', 2), ('deserving', 2), ('lot', 2), ('hifumi', 1), ('liking', 1)]


**hard to do vector similarity because `en_core_web_lg` doesn't have vector values for some of the BA_characters**

also, not sure if the vector values will actually represent the character

to note:
>SpaCy's native models, such as en_core_web_lg, the position of a word in a sentence does not affect its vector representation. These models use static word embeddings, where each word is assigned a fixed vector based on the word itself, regardless of its position or the context in which it appears.

so it's okay to de-dupluicate all the words with the same `Token.text` value

In [25]:
targets = [tk for doc in docs for tk in doc if tk._.ba_characters == target_character]
print(Counter([t.text for t in targets]))

# remove duplicate tokens for targets
dedup = {
    target.text: target for target in targets 
}
targets = [t for t in dedup.values()]

for t in targets:
    assert t.has_vector, f"{t.text} doesn't have a vector value"

print('`cm_original` no vector:')
[print(t) for t in cm_original_1 if not t.has_vector]

vec_sim = {target.text: {tk.text: tk.similarity(target) for tk in cm_original_1 if tk.has_vector} for target in tqdm(targets) if target.has_vector}

Counter({'sensei': 91})
`cm_original` no vector:
nonomi
omagari
eimi
lesgoo
abydos
himari
ooooo
코티코
hitwomen
sanra


  0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
print([k for k in vec_sim])
print(len(vec_sim))
{k: v for k, v in sorted(vec_sim['sensei'].items(), key=lambda item: item[1], reverse=True)}

['sensei']
1


{'sense': 0.605985164642334,
 'happiness': 0.40256187319755554,
 'feel': 0.3986526429653168,
 'impression': 0.3744967579841614,
 'body': 0.3569816052913666,
 'harmony': 0.3567489683628082,
 'love': 0.3563920855522156,
 'respect': 0.35223153233528137,
 'life': 0.3454945385456085,
 'interest': 0.30065616965293884,
 'little': 0.30037540197372437,
 'meant': 0.3002455532550812,
 'like': 0.30020174384117126,
 'logical': 0.2966761589050293,
 'happened': 0.2865977883338928,
 'wanting': 0.28550535440444946,
 'loving': 0.2793347239494324,
 'strong': 0.2783357501029968,
 'chance': 0.2764001488685608,
 'teasing': 0.27630293369293213,
 'tries': 0.2691846787929535,
 'maybe': 0.2691352069377899,
 'work': 0.26856309175491333,
 'moves': 0.26222139596939087,
 'reveal': 0.254867821931839,
 'look': 0.2531861364841461,
 'want': 0.25263577699661255,
 'seeing': 0.25062114000320435,
 'needs': 0.24460053443908691,
 'erotic': 0.2419493943452835,
 'student': 0.24087487161159515,
 'tempting': 0.2396874725818634,


# Using FastText to deal with OOV tokens
above scoring and text2vec has limited vocab so has limit in determining the similarity between words, so going to use FastText(unsupervised training) to make word vectors.

1. combine the preprocesed Doc object to a single text file
2. feed that into the FastText
3. replace the vector value of the prev. Doc as the FastText's vector
4. do similarity tests 

## preprocess

In [27]:
def is_cleanable(tk: spacy.tokens.token.Token):
    # might(?) try and add emoji if that seems plausible
    if (tk.is_alpha and not tk.is_stop and not tk._.is_emoji) or tk.is_sent_end:
        return True
    else:
        return False

def preprocess_tk(tk: spacy.tokens.token.Token):
    if not tk.is_alpha:
        tk = '\n'
    elif tk.is_sent_end:
        tk = tk.lemma_.lower().strip()
        tk += '\n'
    else:
        tk = tk.lemma_.lower().strip()
        tk += ' '
    
    return tk


# preprocessed text for FastText
ft_preprocessed = [preprocess_tk(tk) for doc in docs for tk in doc if is_cleanable(tk)]
print(len(ft_preprocessed))
ft_preprocessed = ''.join(ft_preprocessed)
ft_preprocessed = re.sub(" +", " ", ft_preprocessed) # fixes double(or more) spaces
ft_preprocessed = re.sub("<[^>]*>", "", ft_preprocessed) # remove html tags

print(ft_preprocessed.count('\n'))

ft_path = path.join(cwd, 'ft_preprocessed.txt')
with open(ft_path, 'w', encoding='utf-8') as f:
    f.write(ft_preprocessed)

6376
1343


In [28]:
import fasttext
print(ft_path)
model = fasttext.train_unsupervised(ft_path)

../results/ft_preprocessed.txt


Read 0M words
Number of words:  215
Number of labels: 0
Progress: 100.0% words/sec/thread:   29443 lr:  0.000000 avg.loss:  4.121738 ETA:   0h 0m 0s


In [29]:
model.words

['</s>',
 'year',
 'sensei',
 'new',
 'like',
 'get',
 'pull',
 'happy',
 'good',
 'source',
 'want',
 'time',
 'pixiv',
 'need',
 'arona',
 'look',
 'student',
 'go',
 'twitter',
 'love',
 'banner',
 'himari',
 'cute',
 'chan',
 'yuuka',
 'hoshino',
 'hina',
 'plana',
 'daughter',
 'blue',
 'hope',
 'artist',
 'ny',
 'think',
 'rio',
 'bad',
 'mutsuki',
 'spark',
 'mob',
 'give',
 'lot',
 'thank',
 'let',
 'come',
 'day',
 'aru',
 'girl',
 'see',
 'thing',
 'well',
 'archive',
 'know',
 'wish',
 'right',
 'toki',
 'correction',
 'art',
 'end',
 'nonomi',
 'mika',
 'fuuka',
 'seagull',
 'draw',
 'hifumi',
 'spend',
 'take',
 'damn',
 'stop',
 'bunny',
 'kayoko',
 'probably',
 'read',
 'terror',
 'make',
 'miyu',
 'little',
 'midori',
 'oh',
 'ready',
 'ai',
 'enjoy',
 'black',
 'u',
 'nagisa',
 'money',
 'fire',
 'wait',
 'start',
 'aris',
 'stay',
 'feel',
 'fall',
 'translate',
 'kivoto',
 'help',
 'peak',
 'try',
 'body',
 'big',
 'halo',
 'game',
 'comment',
 'seminar',
 'koyuki',


In [30]:
model.get_nearest_neighbors('sensei')

[(0.8276190161705017, 'senseis'),
 (0.5786563158035278, 'fuuka'),
 (0.5660360455513, '</s>'),
 (0.5455647110939026, 'seagull'),
 (0.5314084887504578, 'student'),
 (0.5086376667022705, 'get'),
 (0.5035670399665833, 'good'),
 (0.5011792778968811, 'saori'),
 (0.49223724007606506, 'care'),
 (0.4729119539260864, 'spark')]